## ダウンロードしたデータの結合

前項でダウンロードしたデータは、日付ごとに分かれているので、1つのファイルにまとめます。

In [12]:
import re
import pandas as pd
#from tqdm import tqdm_notebook as tqdm
from tqdm import tqdm
from os import path
from pathlib import Path

In [21]:
DATA_DIR = 'Z:/自動売買'  # データをダウンロードしたフォルダ
PRICE_DIR = path.join(DATA_DIR, 'data')  # 株価一覧表
STOCK_DATA_DIR = path.join(DATA_DIR, 'japan-all-stock-data')  # 投資指標データ
FIN_RESULTS_DIR = path.join(DATA_DIR, 'japan-all-stock-financial-results')  # 決算・財務・業績データ
OUT_DIR = 'C:/sample/data'  # 結合したデータを保存するフォルダ

In [29]:
price_data = []
file_list = list(Path(PRICE_DIR).glob('*.csv'))  # PRICE_DIR以下にあるcsvファイルのパスをすべて取得する

for file in tqdm(file_list):
    data_ = pd.read_csv(f'file:{file}', encoding='sjis', na_values='-')
    data_.columns = [column.strip() for column in data_.columns]  # カラム名にスペースが混じることがあるので削除
    # timestamp = pd.Timestamp(re.findall(r'\d{8}', file.stem)[0])  # 日時はファイル名から取得
    # price_data.append(data_.assign(日時=timestamp))
    price_data.append(data_)
    break



,Date,Open,Hight,Low,Close,Volume,Adj Close
0,1983/01/04,548.0,555.0,548.0,548.0,182000,498.2
1,1983/01/05,550.0,554.0,550.0,551.0,450000,500.9
2,1983/01/06,553.0,559.0,553.0,555.0,604000,504.6
3,1983/01/07,560.0,564.0,558.0,564.0,1728000,512.7
4,1983/01/08,573.0,582.0,571.0,575.0,3027000,522.7
...,...,...,...,...,...,...,...
9025,2018/11/12,3111.0,3125.0,3092.0,3117.0,3355800,3117.0
9026,2018/11/13,3055.0,3061.0,3020.0,3055.0,4878600,3055.0
9027,2018/11/14,3025.0,3051.0,3021.0,3038.0,4414000,3038.0
9028,2018/11/15,3038.0,3047.0,3022.0,3032.0,2807900,3032.0


In [ ]:
stock_data = []
file_list = list(Path(STOCK_DATA_DIR).glob('*.csv'))  # STOCK_DATA_DIR以下にあるcsvファイルのパスをすべて取得する

for file in tqdm(file_list):
    data_ = pd.read_csv(f'file:{file}', encoding='sjis', na_values='-')
    data_.columns = [column.strip() for column in data_.columns]  # カラム名にスペースが混じることがあるので削除
    timestamp = pd.Timestamp(re.findall(r'\d{8}', file.stem)[0])  # 日時はファイル名から取得
    stock_data.append(data_.assign(日時=timestamp))

stock_data = pd.concat(stock_data).assign(
    日時=lambda x: x['日時'].map(
        lambda elm: pd.Timestamp(pd.to_datetime(elm).date())  # 時刻をすべて00:00:00に合わせる
    ),
    安値日付=lambda x: x['安値日付'].map(
        lambda elm: pd.Timestamp(pd.to_datetime(elm).date())  # 時刻をすべて00:00:00に合わせる
    ),
    高値日付=lambda x: x['高値日付'].map(
        lambda elm: pd.Timestamp(pd.to_datetime(elm).date())  # 時刻をすべて00:00:00に合わせる
    )
)

In [ ]:
daily_data = pd.merge(price_data, stock_data, on=['SC', '時価総額（百万円）', '名称', '市場', '業種', '日時'], how='left')

In [ ]:
financial_data = []
file_list = list(Path(FIN_RESULTS_DIR).glob('*.csv'))
for file in tqdm(file_list):
    data_ = pd.read_csv(f'file:{file}', encoding='sjis', na_values='-')
    data_.columns = [column.strip() for column in data_.columns]
    timestamp = pd.Timestamp(re.findall(r'\d{8}', file.stem)[0])
    financial_data.append(data_.assign(日時=timestamp))

financial_data = pd.concat(financial_data)
financial_data = financial_data.assign(
    日時=lambda x: x['日時'].map(
        lambda elm: pd.Timestamp(pd.to_datetime(elm).date())
    ),
    kessan_tmp=lambda x: x['決算発表日（本決算）'].map(
        lambda elm: pd.Timestamp(pd.to_datetime(elm).date())  # 全角()が扱えないので一時カラムで対処
    )
).drop(columns='決算発表日（本決算）').rename(columns={'kessan_tmp': '決算発表日（本決算）'})[financial_data.columns]  # 列の順番を元に戻す

結合したデータを保存します。pickleという形式で保存することでデータの型などのメタ情報をそのまま保存できます。

In [ ]:
daily_data.to_pickle(path.join(OUT_DIR, 'daily_data.pickle'))
financial_data.to_pickle(path.join(OUT_DIR, 'financial_data_all.pickle'))